# Quantum Phase Estimation

Implementation of phase estimation

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFT
from qiskit_aer import Aer
from collections import Counter
import math

In [2]:
def query_function(marked_state, qc, qr):

    """
    Apply the Grover query operator for a given marked state.

    Parameters:
    marked_state (str): The marked state.
    qc (QuantumCircuit): The quantum circuit.
    qr (QuantumRegister): The quantum register.
    """

    # apply X gate to qubits if marked_state has a zero 
    for i in range(len(marked_state)):
        if marked_state[i] == '0':
            qc.x(qr[len(marked_state)-1-i])

    # apply multi controlled X gate to get sign of -1 for the marked_state 
    qc.mcx(control_qubits=list(range(0,qr.size-1)), target_qubit=qr[qr.size-1])
    
    # apply X gate again to revert 
    for i in range(len(marked_state)):
        if marked_state[i]=='0':
            qc.x(qr[len(marked_state)-1-i])

In [3]:
def diffusion_operator(qc, qr):
    """
    Apply the diffusion operator.

    Parameters:
    qc (QuantumCircuit): The quantum circuit.
    qr (QuantumRegister): The quantum register.
    """

    # this implementation produces the diffusion operator, but without the minus 1 sign in -HS_0H

    # apply H and X gate
    for i in range(qr.size-1):
        qc.h(qr[i])
        qc.x(qr[i])

    # apply multi controlled Z gate to get sign of -1 for the all 1 state. 
    # by applying X gates before and after this operation, the sign of -1 gets applied to the all 0 state instead 
    qc.h(qr[qr.size-2])
    qc.mcx(control_qubits=list(range(0,qr.size-2)), target_qubit=qr[qr.size-2])
    qc.h(qr[qr.size-2])

    # apply X and H gate
    for i in range(qr.size-1):
        qc.x(qr[i])
        qc.h(qr[i])

In [4]:
def control_u_gate(power, marked_states):
    """
    Construct a controlled Grover operator.

    Parameters:
    power (int): The power of the Grover operator.
    marked_states (list): List of marked states.

    Returns:
    Gate: Controlled Grover operator.
    """
    
    n = len(marked_states[0])
    qr = QuantumRegister(n+1)
    qc = QuantumCircuit(qr)
    
    # create grover operator raised to powers of 2
    for _ in range(2**power):
        for marked_state in marked_states:
            query_function(marked_state, qc, qr)    
        diffusion_operator(qc, qr)

    # convert the quantum circuit to a gate and make it a controlled gate
    operator = qc.to_gate().control()
    
    return operator    

Build circuit

In [5]:
# define marked states and the number of qubits
marked_states = ['100', '101', '111']
m = 5 # ancilla qubits
n = len(marked_states[0]) #qubits to describe the eigenvector

# create quantum and classical registers
qr = QuantumRegister(n+m+1)
cr = ClassicalRegister(m)
qc = QuantumCircuit(qr,cr)

# initialize the ancilla qubit to the 1 state
qc.x(qr[qr.size-1])

# put all qubits in superposition
qc.h(qr[0:qr.size])

#m = cr.size

# apply control U gates of grover operator
for i in range(m):
    indices = [i]
    indices+=list(range(m,qr.size))
    qc.append(control_u_gate(power=i, marked_states=marked_states), indices)

# apply inverse QFT on ancilla qubits
qc.append(QFT(m).inverse(), qr[0:m])
qc.measure(qr[0:m], cr)
qc.draw()

┌───┐                                                     »
q1_0: ┤ H ├────────────■────────────────────────────────────────»
      ├───┤            │                                        »
q1_1: ┤ H ├────────────┼───────────────■────────────────────────»
      ├───┤            │               │                        »
q1_2: ┤ H ├────────────┼───────────────┼───────────────■────────»
      ├───┤            │               │               │        »
q1_3: ┤ H ├────────────┼───────────────┼───────────────┼────────»
      ├───┤            │               │               │        »
q1_4: ┤ H ├────────────┼───────────────┼───────────────┼────────»
      ├───┤     ┌──────┴───────┐┌──────┴───────┐┌──────┴───────┐»
q1_5: ┤ H ├─────┤0             ├┤0             ├┤0             ├»
      ├───┤     │              ││              ││              │»
q1_6: ┤ H ├─────┤1             ├┤1             ├┤1             ├»
      ├───┤     │  circuit-165 ││  circuit-242 ││  circuit-358 │»
q1_7: ┤ H ├─────┤2             ├┤2             ├┤2             ├»
      ├───┤┌───┐│              ││              ││              │»
q1_8: ┤ X ├┤ H ├┤3             ├┤3             ├┤3             ├»
      └───┘└───┘└──────────────┘└──────────────┘└──────────────┘»
c0: 5/══════════════════════════════════════════════════════════»
                                                                »
«                                      ┌───────┐┌─┐            
«q1_0: ────────────────────────────────┤0      ├┤M├────────────
«                                      │       │└╥┘┌─┐         
«q1_1: ────────────────────────────────┤1      ├─╫─┤M├─────────
«                                      │       │ ║ └╥┘┌─┐      
«q1_2: ────────────────────────────────┤2 IQFT ├─╫──╫─┤M├──────
«                                      │       │ ║  ║ └╥┘┌─┐   
«q1_3: ───────■────────────────────────┤3      ├─╫──╫──╫─┤M├───
«             │                        │       │ ║  ║  ║ └╥┘┌─┐
«q1_4: ───────┼───────────────■────────┤4      ├─╫──╫──╫──╫─┤M├
«      ┌──────┴───────┐┌──────┴───────┐└───────┘ ║  ║  ║  ║ └╥┘
«q1_5: ┤0             ├┤0             ├──────────╫──╫──╫──╫──╫─
«      │              ││              │          ║  ║  ║  ║  ║ 
«q1_6: ┤1             ├┤1             ├──────────╫──╫──╫──╫──╫─
«      │  circuit-552 ││  circuit-902 │          ║  ║  ║  ║  ║ 
«q1_7: ┤2             ├┤2             ├──────────╫──╫──╫──╫──╫─
«      │              ││              │          ║  ║  ║  ║  ║ 
«q1_8: ┤3             ├┤3             ├──────────╫──╫──╫──╫──╫─
«      └──────────────┘└──────────────┘          ║  ║  ║  ║  ║ 
«c0: 5/══════════════════════════════════════════╩══╩══╩══╩══╩═
«                                                0  1  2  3  4

In [6]:
# select the qasm simulator as the backend
simulator = Aer.get_backend("qasm_simulator")

# transpile the circuit for the selected simulator
qc_transpiled = transpile(qc, simulator)

# run the transpiled circuit on the simulator
job = simulator.run(qc_transpiled, shots=2000) 

# get results
result = job.result()
counts = result.get_counts(qc)
most_frequent = max(counts, key=counts.get)

# convert the binary string into an integer
int_value = int(most_frequent, 2)

In [7]:
# estimated value of theta 
print(math.pi*int_value/(2**m)+math.pi/2)
print(math.pi*int_value/(2**m)-math.pi/2)

3.8288160465625602
0.6872233929727671


In [8]:
# correct value of theta
print(math.asin((len(marked_states)/2**len(marked_states[0]))**0.5))

0.6590580358264089


In [9]:
# estimated applitude
print(math.sin(math.pi*int_value/(2**m)+math.pi/2)**2)
print(math.sin(math.pi*int_value/(2**m)-math.pi/2)**2)

0.40245483899193557
0.40245483899193574


In [10]:
# correct amplitude
(len(marked_states)/2**len(marked_states[0]))

0.375

In [11]:
# number of solutions
math.sin(math.pi*int_value/(2**m)-math.pi/2)*math.sin(math.pi*int_value/(2**m)-math.pi/2)*2**len(marked_states[0])

3.219638711935486